In [1]:
import wrds 

WRDS Thomson Reuters Mutual Funds: https://wrds-www.wharton.upenn.edu/pages/get-data/thomson-reuters/ 

WRDS CRSP Mutual Funds: https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_q_mutualfunds/ 

Username: lyknguyen 
Password: Khanhly_1609 

In [2]:
conn = wrds.Connection() 

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# conn.list_tables(library='crsp') 
# conn.list_tables(library='tr_mutualfunds') 

In [7]:
import pandas as pd 

In [5]:
# DATA SOURCE 1.1 - CRSP Mutual Funds (fund_summary2)
# conn.get_table(library='crsp',table='fund_summary2',obs=5).columns 

In [4]:
# DATA SOURCE 1.1 - CRSP Mutual Funds (fund_summary2)
# conn.get_table(library='crsp',table='fund_summary2',obs=5).columns 

# Takes 1 minute to run 
crspfundsum = conn.raw_sql(
    """
    select crsp_fundno,ticker,caldt,crsp_obj_cd,si_obj_cd,mgmt_cd,fund_name,index_fund_flag,retail_fund,inst_fund,lipper_class_name
    from crsp.fund_summary2 
    where caldt <= '2018-12-31'
    order by crsp_fundno, caldt
    """, 
    date_cols=['caldt']
) 

crspfundsum['crsp_fundno'] = crspfundsum['crsp_fundno'].astype(int) # convert to integer 
crspfundsum['year'] = crspfundsum['caldt'].dt.year  # extract the year 

# sort and then duplicates drop fundno and year, keep the last occurrence of each duplicated row 
crspfundsum = crspfundsum.sort_values(by=['crsp_fundno','caldt']).drop_duplicates(subset=['crsp_fundno','year'],keep='last') 

In [8]:
# DATA SOURCE 1.2 - CRSP Mutual Funds (monthly_tna_ret_nav) 
# also takes about 1 minute to run 

monthly_tna_ret_nav = conn.raw_sql(
    """
    select crsp_fundno,caldt,mret,mtna,mnav
    from crsp.monthly_tna_ret_nav
    where caldt >= '1980-01-01'
    AND caldt <= '2019-12-31'
    order by crsp_fundno, caldt
    """, 
    date_cols=['caldt']
)
monthly_tna_ret_nav['crsp_fundno'] =monthly_tna_ret_nav['crsp_fundno'].astype(int)  # convert to integer 
monthly_tna_ret_nav = monthly_tna_ret_nav.dropna()   # drop missing values 
monthly_tna_ret_nav = monthly_tna_ret_nav.sort_values(by=['crsp_fundno', 'caldt'])
monthly_tna_ret_nav["ym"] = pd.to_datetime(monthly_tna_ret_nav['caldt'], format='%Y-%m').dt.to_period('M')
#monthly_tna_ret_nav = monthly_tna_ret_nav.drop(columns=['caldt'])
#monthly_tna_ret_nav['year'] = monthly_tna_ret_nav.ym.dt.year 